<a href="https://colab.research.google.com/github/larabjork/amazon-vine-analysis/blob/main/Amazon_Reviews_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://ppa.launchpad.net/c2d4u.te

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-01-08 22:39:44--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.2’

postgresql-42.2.16. 100%[===================>] 979.38K  5.47MB/s    in 0.2s    

2023-01-08 22:39:45 (5.47 MB/s) - ‘postgresql-42.2.16.jar.2’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_02.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12076615| RQ58W7SMO911M|0385730586|     122662979|Sisterhood of the...|           Books|          4|            2|          3|   N|                N|this book was a g...|this boook was a ...| 2005-10-14|
|         US|   12703090|  RF6IUKMGL8SF|0811828964|      56191234|The Bad Girl's Gu...|           Books|          3|    

### Create DataFrames to match tables

In [ ]:
from pyspark.sql.functions import to_date, count
# Read in the Review dataset as a DataFrame


In [ ]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").agg(count("*")).withColumnRenamed("count(1)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   27314089|             1|
|    6711334|             1|
|   44525866|             4|
|   47461997|             1|
|   42560427|             1|
|   35225823|             1|
|   52526865|            20|
|    1954060|             1|
|   34202730|             3|
|   40014361|             1|
|   42719693|             1|
|   27252006|             1|
|   48297144|             1|
|   12204397|             1|
|   17090175|             2|
|   13352125|             9|
|   46958825|            28|
|     134138|             1|
|     283456|             1|
|   42847010|             2|
+-----------+--------------+
only showing top 20 rows



In [ ]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(['product_id', 'product_title']).drop_duplicates()
products_df.show()


+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B003WZXSWM|Grover 205BC Mini...|
|B00ASHQQKI|SUPERNIGHT 5050 1...|
|B000T4BC1I|Aulos A211A/A511B...|
|B005I0HTCY|Neewer Black 3.5m...|
|B00JJ18VIE|Reliable Hardware...|
|B00US77MXI|Pick Geek Guitar ...|
|B00S5O7HZO|Glory B Flat Clar...|
|B007CPU1A0|Meinl Headliner T...|
|B0002E3D9Y|PACK A STAND Clar...|
|B00J3W1F96|Zildjian 5A Acorn...|
|B0006NMUJU|Shure Foam Windsc...|
|B0018SJLFS|Tung-Sol 12AX7 Pr...|
|B00N0HXT94|Akai Professional...|
|B0006GRRQI|Gator G-PG ACOUST...|
|B005DJOI8I|Neewer 3.5mm Hand...|
|B000WS1QC6|Yamaha PA130 120 ...|
|B00IEQ4CVK|4/4 Size Electric...|
|B0080GZPRY|VocoPro CARRY-OKE...|
|B00I4ZMW2C|San Fransisco Gia...|
|B000F9NBZC|Tapco by Mackie M...|
+----------+--------------------+
only showing top 20 rows



In [ ]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
from pyspark.sql.functions import col
review_id_df = df.select(['review_id', 'customer_id', 'product_id', 'product_parent', 'review_date',to_date(col('review_date'),'yyyy-MM-dd').alias("date")])
# review_id_df.show()
# review_id_df.printSchema()
review_id_df_drop = review_id_df.drop('review_date')
review_id_clean_df = review_id_df_drop.withColumnRenamed("date", "review_date")


review_id_dedupe = review_id_clean_df.select(['review_id', 'customer_id', 'product_id', 'product_parent', 'review_date']).drop_duplicates()
review_id_dedupe.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1TNWRKIVHVYOV|   51964897|0262181533|     598678717| 2005-10-14|
|R327Y8FVALBGWW|   24513242|0785263705|     407388798| 2005-10-14|
|R27TR8B9R48TCJ|   38003326|1572704608|      41708890| 2005-10-13|
| REGTR7OIJIKTC|   12085394|B0006QSZ64|     135091908| 2005-10-13|
|R29P5P88TU51WT|   12144985|1559707712|     958157787| 2005-10-13|
|R2ERZFUM15XKAF|   36097625|083863561X|     655412122| 2005-10-13|
|R1QM96O7JCZVY0|   50985049|0060925531|     698128879| 2005-10-13|
|R3VQ50AKWDJS2I|   12086273|0814473016|     597072864| 2005-10-13|
|R2Q49FPW8ONB0P|   13138109|0325005125|     392789664| 2005-10-13|
|R30E74BSA9TTOG|   16402611|0837601088|     703998792| 2005-10-13|
|R13YRU1US1Q9QL|   44892864|0385484550|     870146650| 2005-10-13|
| RJ4PXKR6C5I8A|   38882660|0195183010|     239260223| 2005-10

In [ ]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id', 'star_rating','helpful_votes','total_votes','vine','verified_purchase'])
vine_df.show()


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RMDCHWD0Y5OZ9|          3|            0|          1|   N|                N|
| RZSL0BALIYUNU|          5|            0|          0|   N|                Y|
| RIZR67JKUDBI0|          3|            0|          1|   N|                Y|
|R27HL570VNL85F|          5|            0|          0|   N|                Y|
|R34EBU9QDWJ1GD|          5|            0|          0|   N|                Y|
|R1WCUI4Z1SIQEO|          5|            0|          0|   N|                N|
| RL5LNO26GAVJ1|          2|            3|          4|   N|                Y|
|R3GYQ5W8JHP8SB|          5|            0|          0|   N|                Y|
|R30SHYQXGG5EYC|          5|            0|          0|   N|                Y|
|R14YLXA56NP51I|          5|            1|          1|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql:<connection-string>:5432/<database-name"
config = {"user":"postgres", 
          "password": "<password>", 
          "driver":"org.postgresql.Driver"}



In [ ]:
# Write review_id_df to table in RDS
review_id_dedupe.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)